# Points of Dispense
#### Minimize maximum distance and vary number of PODs
Maxwell Kennady, Nora Murray, Elizabeth Speigle

In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

### Optimization

Read in data from files

In [3]:
distances = pd.read_csv('data/OD_Pairs_Distances.csv')
population = pd.read_excel('data/BG_master.xlsx')

In [4]:
dist_miles = distances.pivot(index='block_group', columns='pod_id', values='Miles')
dist_time = distances.pivot(index='block_group', columns='pod_id', values='TravelTime')

In [5]:
dist = dist_miles.values
N = population['population'].values
prop = population['flood'].values

Create indices for block groups and PODs

In [6]:
blocks = range(len(N))
pods = range(len(dist[0]))

Initialize model for POD locations

In [10]:
pods_open = range(1, 48)
pods_open_results = {}

In [11]:
for i in pods_open: 
    ### Initialize model for POD locations
    m = gp.Model('POD_locations')
    
    ### Add decision variables
    x = m.addVars(pods, vtype=GRB.BINARY, name='x')
    y = m.addVars(blocks, pods, vtype=GRB.BINARY, name='x')
    
    ### Set objective to minimize maximum distance
    z = m.addVar(vtype=GRB.CONTINUOUS, name='z')
    m.setObjective(z, GRB.MINIMIZE)
    
    ### Add constraints
    #  y[i,j] can only be 1 if x[i] is also 1, meaning POD i is opened
    m.addConstrs((y[j, i] <= x[i] for i in pods for j in blocks), name='y_if_x')
    
    # each block group must be assigned one shelter
    m.addConstrs((gp.quicksum(y[j, i] for i in pods) == 1
             for j in blocks), name='all_blocks_assigned')
    
    # number of PODs opened must be less than num_pods
    m.addConstr((gp.quicksum(x[i] for i in pods) <= i), name='pods_opened')
    
    # maximum distance
    m.addConstrs((z >= dist[j, i] * y[j, i] for i in pods for j in blocks), 
             name='Distance_Below_Max')
    
    ### Optimize
    print('Solving for:', i, 'PODs')
    m.optimize()
    
    ### Analysis
    block_pod_list = [[j,i] for j in blocks for i in pods if (y[j, i].x==1) & (prop[j]==1)]
    pods_open_results[i] = block_pod_list

Solving for: 1 PODs
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 104501 rows, 51748 columns and 258547 nonzeros
Model fingerprint: 0x78e63881
Variable types: 1 continuous, 51747 integer (51747 binary)
Coefficient statistics:
  Matrix range     [8e-02, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 1.11s
Presolved: 104501 rows, 51748 columns, 258547 nonzeros
Variable types: 1 continuous, 51747 integer (51747 binary)
Found heuristic solution: objective 30.4291814

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   52127    6.6619546e+00   6.113019e-01   1.103004e+08      5s
   55337    1.5900169e+01   3.440419e-01   1.777139e+08     10s
   57917    3.0402104e+01   6.803583e-04   1.330315e+08     15s
   63687    1.4910896e+01   0.000000e+00   5.165374e+02 

In [13]:
pods_flood_minmax = open('pods_flood_minmax','wb')
pickle.dump(pods_open_results, pods_flood_minmax)
pods_flood_minmax.close()